In [1]:
import os
import numpy as np
from netCDF4 import Dataset

In [2]:
def list_nc_files(directory):
    return [f for f in os.listdir(directory) if f.endswith('.nc')]

def extract_time_step(file_path, time_step):
    with Dataset(file_path, 'r') as nc_file:
        # Assuming variables are stored in a dict-like structure
        variables = {var: nc_file.variables[var][time_step, :, :] for var in nc_file.variables}
    return variables

def add_geopotential(file_path, num_days):
    with Dataset(file_path, 'r') as nc_file:
        # Assuming 'z' is the variable of interest in the geopotential file
        geopotential = nc_file.variables['z'][:num_days, :, :]
    return geopotential

def save_new_file(output_path, stacked_data):
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with Dataset(output_path, 'w', format='NETCDF4') as nc_file:
        # Create dimensions
        nc_file.createDimension('time', stacked_data.shape[0])
        nc_file.createDimension('variable', stacked_data.shape[1])
        nc_file.createDimension('lat', stacked_data.shape[2])
        nc_file.createDimension('lon', stacked_data.shape[3])

        # Create variable
        data_var = nc_file.createVariable('data', np.float32, ('time', 'variable', 'lat', 'lon'))
        data_var[:, :, :, :] = stacked_data

def verify_file(output_path, expected_shape):
    with Dataset(output_path, 'r') as nc_file:
        data_var = nc_file.variables['data']
        actual_shape = data_var.shape
        return actual_shape == expected_shape


In [ ]:
input_directory = '/data_aip05/gsaliou/era5/local/DATA_ERA5/new/'
geopotential_file = '/data_aip05/gsaliou/era5/local/DATA_ERA5/new/oro/geopotential_0.25_local.nc'
output_directory = '/data_aip05/gsaliou/local_1day/'
variables = ['tcwv', 'msl', 'sp', 't2m', 'skt', 'tisr', 'z1', 'z2', 'z3', 'z4', 'z5', 'z6', 'z7', 'z8', 'z9', 'z10', 'z11', 'z12', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'q11', 'q12', 'u1', 'u2', 'u3', 'u4', 'u5', 'u6', 'u7', 'u8', 'u9', 'u10', 'u11', 'u12', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12']

In [ ]:
nc_files = list_nc_files(input_directory)

for i in nc_files:
    yearly_data = []
    # Year from the file name
    year = int(i.split('/')[-1].split('.')[0])
    if year % 4 == 0:
        num_days = 366
    else:
        num_days = 365 
        file_path = os.path.join(input_directory, i)
        daily_data = [extract_time_step(file_path, 12 * day) for day in range(num_days)]
        daily_data = np.stack([np.stack([v for v in day.values()], axis=0) for day in daily_data], axis=0)
        yearly_data.append(daily_data)

        stacked_data = np.stack(yearly_data, axis=0)

        geopotential_data = add_geopotential(geopotential_file, num_days)
        stacked_data_with_geopotential = np.concatenate([stacked_data, geopotential_data[:, np.newaxis, :, :]], axis=1)
        output_file = os.path.join(output_directory,year,'.nc')
        save_new_file(output_file, stacked_data_with_geopotential)

expected_shape = (num_days, 67, 121, 81)
if verify_file(output_file, expected_shape):
    print(f"New file saved correctly with shape {expected_shape}.")
else:
    print("Error: The new file does not have the expected shape.")